In [182]:
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [183]:
def load_systems_dataframe(ulx_only=False, beamed=False, half_opening_l_45=False):
    systems_df_path = Path('../data/processed/all_systems_df.csv')
    df = pd.read_csv(systems_df_path)
    if ulx_only:
        df = df[df['Lx'] > 1E39]
    if beamed:
        df = df[df['b'] < 1]
    if half_opening_l_45:
        df = df[df['theta_half_deg'] < 45]
        
    df = df.drop(['Unnamed: 0'], axis=1)
    return df

In [184]:
Z = 0.02
systems_df = load_systems_dataframe(True, False, False)
if Z == 'all':
    systems_df = systems_df
else:
    systems_df = systems_df[systems_df['Z']==Z]

In [185]:
curve_classifications = pd.read_csv('../data/processed/curve_classications.csv')

In [186]:
# Fast BH sampler

In [187]:
def create_keys(selected_systems, selected_dincls, selected_inclinations,):
    a = np.core.defchararray.add(selected_systems.astype(str), '-')
    b = np.core.defchararray.add(selected_dincls.astype(str), '-')
    keys = np.core.defchararray.add(a,b)
    keys = np.core.defchararray.add(keys,selected_inclinations.astype(str))
    return keys

In [188]:
def create_classification_dict():
    # Classication dict
    keys = (curve_classifications['system_id'].astype(str) +
            '-' + curve_classifications['dincl'].astype(str) +
            '-' + curve_classifications['inclination'].astype(str)).values
    classications = curve_classifications['classification'].values
    class_dict = dict(zip(keys,classications))
    return class_dict

class_dict = create_classification_dict()

In [189]:
bh_systems = systems_df[systems_df['is_bh'] == 1].index
ns_systems = systems_df[systems_df['is_bh'] == 0].index

In [190]:
res = []
number_of_repeats = 10000
for i in tqdm(range(number_of_repeats)):
    for bh_ratio in np.arange(0,1.05,0.05):
        ns_ratio = 1 - bh_ratio

        bh_weights = [bh_ratio/len(bh_systems)]*len(bh_systems)
        ns_weights = [ns_ratio/len(ns_systems)]*len(ns_systems)

        selected_systems = np.random.choice([*bh_systems, *ns_systems], size=500, p=[*bh_weights, *ns_weights])
        selected_dincls = np.random.randint(0,46, size=500)
        selected_inclinations = np.random.randint(0,91, size=500)
        selected_keys = create_keys(selected_systems, selected_dincls, selected_inclinations)

        res_classications = [class_dict.get(key) for key in selected_keys]

        # None systems correspond to opening angles > 45 and are considered alive
        N_alive = res_classications.count('alive') + res_classications.count(None)
        N_transient = res_classications.count('transient')
        N_dead = res_classications.count('dead')

        res.append([bh_ratio, N_alive, N_transient, N_dead])

100%|██████████| 10000/10000 [07:00<00:00, 23.81it/s]


In [191]:
res_df = pd.DataFrame(res, columns=['bh_ratio', 'alive', 'transient','dead'])

In [192]:
savepath = '../data/interim/bh_ns_sampling_results/'
filename = f'Z = {Z}.csv'
#res_df.to_csv(savepath+filename, index=False)
res_df.to_csv(savepath+filename, mode='a', header=False, index=False)